In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

TRAIN_DIR = "/Users/jamesjoseph/Jamezzz/Final project/IntoTheProject/Indian_currency_dataset/training"

IMAGE_SIZE = (224, 224)  
CLASSES = ["10", "20", "50", "100", "200", "500"]  # Denominations


def load_images_from_directory(directory):
    images = []
    labels = []
    
    for label in CLASSES:
        folder_path = os.path.join(directory, label)  
        if not os.path.exists(folder_path):
            print(f"Warning: Folder for denomination {label} not found!")
            continue
        
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Warning: Could not read {img_path}")
                continue

            img = cv2.resize(img, IMAGE_SIZE)
            images.append(img)
            labels.append(CLASSES.index(label)) 

    images = np.array(images) / 255.0
    labels = to_categorical(labels, num_classes=len(CLASSES))  # One-hot encoding

    return images, labels
images,labels=load_images_from_directory(TRAIN_DIR)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


/Users/jamesjoseph/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Corrupt JPEG data: premature end of data segment


Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


In [15]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load pre-trained model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False
model = Sequential([
    base_model,
    Flatten(),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dense(256, activation="relu"),
    Dense(len(CLASSES), activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    epochs=20)

Epoch 1/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 31s 411ms/step - accuracy: 0.2610 - loss: 4.6035 - val_accuracy: 0.5009 - val_loss: 1.2619
Epoch 2/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 24s 349ms/step - accuracy: 0.5258 - loss: 1.2352 - val_accuracy: 0.6216 - val_loss: 1.0220
Epoch 3/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 24s 345ms/step - accuracy: 0.6186 - loss: 1.0074 - val_accuracy: 0.6252 - val_loss: 1.0595
Epoch 4/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 25s 355ms/step - accuracy: 0.6349 - loss: 0.9707 - val_accuracy: 0.6453 - val_loss: 0.9881
Epoch 5/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 24s 351ms/step - accuracy: 0.7194 - loss: 0.7845 - val_accuracy: 0.6837 - val_loss: 0.8846
Epoch 6/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 25s 357ms/step - accuracy: 0.7323 - loss: 0.7503 - val_accuracy: 0.7130 - val_loss: 0.8254
Epoch 7/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 24s 352ms/step - accuracy: 0.7361 - loss: 0.7252 - val_accuracy: 0.7477 - val_loss: 0.7362
Epoch 8/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 24s 353ms/step - accuracy: 0.7507 - loss: 0.6940 - val_accu

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 241ms/step - accuracy: 0.7674 - loss: 0.7969
Test Accuracy: 78.06%


In [11]:
model.save("currency_detection_model.h5")